<a href="https://colab.research.google.com/github/bobthetitan7/Portfolio_Site/blob/main/NWP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras import Model
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import re
import seaborn as sns

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from absl import logging
from google.colab import files
import nltk
nltk.download('punkt')
from collections import Counter

print("imported necessary libaries")

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

In [ ]:
# uplaod text document for testing
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
alice = 'Alice Adventure.txt'
beyond = 'Beyond Good and Evil.txt'
pride = 'Pride and Prejudice.txt'
gatby = 'Gatsby.txt'
cities = 'The Tale of two Cities.txt'

alice = open(alice).read().lower()
beyond = open(beyond).read().lower()
pride = open(pride).read().lower()
gatby = open(gatby).read().lower()
cities = open(cities).read().lower()

text = alice + beyond + pride + gatby + cities
# you can change the text to lower ram usage


In [ ]:
text_parse = nltk.tokenize.sent_tokenize(text)
words = []
word_process = []
for i in range(len(text_parse)):
    text_parse[i] = text_parse[i].replace('\n',' ')
    text_parse[i] = text_parse[i].replace('  ',' ')
    text_parse[i] = text_parse[i].replace('!','')
    text_parse[i] = text_parse[i].replace('.','')
    text_parse[i] = text_parse[i].replace(',','')
    text_parse[i] = text_parse[i].replace('?','')
    text_parse[i] = text_parse[i].replace(':','')
    text_parse[i] = text_parse[i].replace(';','')
    words_in_parse = text_parse[i].split(' ')
    word_process.append(words_in_parse)
    words = words + words_in_parse
    


sentence_process = text_parse
test_case = []
messages = []
keys = []

for i in range(len(sentence_process)):
    while(len(word_process[i]) >= 4):
        first_three = ' '.join(word_process[i][:3])
        messages.append(first_three)
        keys.append(word_process[i][3])
        pairing = [first_three, word_process[i][3]]
        del word_process[i][0]
        test_case.append(pairing)


In [ ]:
vocab_size = len(Counter(words)) + 1

vocab_size

In [ ]:
values = array(keys)

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values) 

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)


In [ ]:
onehot_encoded.shape

In [ ]:
encoding = embed(messages).numpy()


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(encoding, onehot_encoded, test_size=0.30, random_state=42)

encoding = 0
onehot_encoded = 0

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
X_train = X_train.reshape(30763, 1, 512)
X_test = X_test.reshape(13185, 1, 512)

In [ ]:
X_train.shape

In [ ]:
def plot_curve(epochs, hist, metrics):

  plt.figure()
  plt.xlabel("Epoch")
  plt.ylabel("Value")

  for m in metrics:
    x = hist[m]
    plt.plot(epochs[1:], x[1:], label=m)

  plt.legend()

print("Loaded the plot_curve function.")

In [ ]:
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

def create_model(learning_rate):
  
  model = tf.keras.models.Sequential()

  # 2 LSTM layer with a dropout of 0.2
  model.add(LSTM(1024, dropout=0.2, return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(l=0.00001)))

  model.add(LSTM(1024, dropout=0.2, return_sequences=False, kernel_regularizer=tf.keras.regularizers.l2(l=0.00001)))

  model.add(Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.00001)))
  
  
  # softmax layer
  model.add(tf.keras.layers.Dense(units=vocab_size, activation='softmax'))    

                           
  model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
                loss="categorical_crossentropy",
                metrics=['accuracy'])
  
  return model    

In [ ]:
# def loss(labels, logits):
#    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
#
# def perplexity(labels, logits):
#    cross_entropy = loss(labels, logits)
#    perplexity = tf.keras.backend.exp(cross_entropy)
#    return perplexity

In [ ]:
def train_model(model, x_train, y_train, epochs,
                batch_size=None, validation_split=0.1):

  history = model.fit(x_train, y_train, batch_size=batch_size,
                      epochs=epochs, shuffle=True, 
                      validation_split=validation_split)
 

  epochs = history.epoch
  hist = pd.DataFrame(history.history)


  return epochs, hist    

In [ ]:
learning_rate = 0.001
epochs = 30
batch_size = 1000

validation_split = 0.2



my_model = None

my_model = create_model(learning_rate)

epochs, hist = train_model(my_model, X_train, y_train, 
                           epochs, batch_size, validation_split)



list_of_metrics_to_plot = ['categorical_accuracy']
plot_curve(epochs, hist, list_of_metrics_to_plot)

print("\n Evaluate the new model against the test set:")
my_model.evaluate(X_test, y_test, batch_size=batch_size)